In [ ]:
import copy

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from tqdm.auto import tqdm
from scipy.stats import linregress
import xarray as xr

In [ ]:
scenarios = ['1pct-branch']

In [ ]:
conc_df = pd.read_csv('../data/1pctCO2_concentrations.csv', index_col=0)

In [ ]:
conc_df

In [ ]:
fair_params_df = pd.read_csv('../data/calibrated_constrained_parameters_1.4.1.csv', index_col=0)

In [ ]:
species = ['CO2', 'CH4', 'N2O']
conc_properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
emis_properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# do one at a time
branch = {}
temperature = {}
ohc = {}
toa = {}
carbon_uptake = {}
concentration = {}
erf = {}

ecs = np.ones(841) * np.nan
tcr = np.ones(841) * np.nan

for ce in [750, 1000, 2000]:
    # 1150 is 140 + 1000 + 10: spin up is max. 140 years, we want to run for 1000, and Sofia might take 20 year means so need an extra 10 at end
    branch[ce] = np.zeros(841, dtype=int)
    temperature[ce] = np.ones((1150, 841)) * np.nan
    ohc[ce] = np.ones((1150, 841)) * np.nan
    toa[ce] = np.ones((1150, 841)) * np.nan
    carbon_uptake[ce] = np.ones((1150, 841)) * np.nan   # cumulative emissions minus airborne emissions
    concentration[ce] = np.ones((1150, 841)) * np.nan
    erf[ce] = np.ones((1150, 841)) * np.nan

for idx, iconf in tqdm(enumerate(fair_params_df.index), total=841):
    # 140 year 1pct CO2
    fc = FAIR()
    fc.define_time(0, 140, 1)
    fc.define_configs(list(fair_params_df.index[0:1]))
    fc.define_scenarios(scenarios)
    fc.define_species(species, conc_properties)
    fc.allocate()
    fc.concentration.loc[dict(specie='CH4')] = 808.2490285
    fc.concentration.loc[dict(specie='N2O')] = 273.021047
    fc.concentration.loc[dict(specie='CO2')] = conc_df.values[:,None]
    fc.fill_species_configs()
    fill(fc.climate_configs['ocean_heat_capacity'], fair_params_df.loc[iconf,'ocean_heat_capacity[0]':'ocean_heat_capacity[2]'])
    fill(fc.climate_configs['ocean_heat_transfer'], fair_params_df.loc[iconf,'ocean_heat_transfer[0]':'ocean_heat_transfer[2]'])
    fill(fc.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[iconf,'deep_ocean_efficacy'])
    fill(fc.climate_configs['gamma_autocorrelation'], fair_params_df.loc[iconf,'gamma_autocorrelation'])
    fill(fc.climate_configs['stochastic_run'], False)
    fill(fc.species_configs['iirf_0'], fair_params_df.loc[iconf, 'iirf_0[CO2]'], specie='CO2')
    fill(fc.species_configs['iirf_airborne'], fair_params_df.loc[iconf, 'iirf_airborne[CO2]'], specie='CO2')
    fill(fc.species_configs['iirf_uptake'], fair_params_df.loc[iconf, 'iirf_uptake[CO2]'], specie='CO2')
    fill(fc.species_configs['iirf_temperature'], fair_params_df.loc[iconf, 'iirf_temperature[CO2]'], specie='CO2')
    fill(fc.species_configs["forcing_scale"], fair_params_df.loc[iconf, "forcing_scale[CO2]"], specie='CO2')
    fill(fc.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
    fill(fc.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
    fill(fc.species_configs['baseline_concentration'], 273.021047, specie='N2O')
    fill(fc.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
    fill(fc.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
    fill(fc.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')
    initialise(fc.concentration, fc.species_configs['baseline_concentration'])
    initialise(fc.forcing, 0)
    initialise(fc.temperature, 0)
    initialise(fc.airborne_emissions, 0)
    initialise(fc.cumulative_emissions, 0)
    initialise(fc.ocean_heat_content_change, 0)
    fc.run(progress=False)

    for ce in [750, 1000, 2000]:
        branch[ce][idx] = np.argmax(fc.cumulative_emissions[:,0,0,0].data/3.664>ce)

    for ce in [750, 1000, 2000]:
        # re-run C-driven to the three branch points and save out restarts
        fc = FAIR()
        fc.define_time(0, branch[ce][idx], 1)
        fc.define_scenarios([f'1pct-branch-{ce}PgC'])
        fc.define_configs([iconf])
        fc.define_species(species, conc_properties)
        fc.allocate()
        fc.concentration.loc[dict(specie='CH4')] = 808.2490285
        fc.concentration.loc[dict(specie='N2O')] = 273.021047
        fc.concentration.loc[dict(specie='CO2')] = conc_df.values[:branch[ce][idx]+1,None]
        fc.fill_species_configs()
        fill(fc.climate_configs['ocean_heat_capacity'], fair_params_df.loc[iconf,'ocean_heat_capacity[0]':'ocean_heat_capacity[2]'])
        fill(fc.climate_configs['ocean_heat_transfer'], fair_params_df.loc[iconf,'ocean_heat_transfer[0]':'ocean_heat_transfer[2]'])
        fill(fc.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[iconf,'deep_ocean_efficacy'])
        fill(fc.climate_configs['gamma_autocorrelation'], fair_params_df.loc[iconf,'gamma_autocorrelation'])
        fill(fc.climate_configs['stochastic_run'], False)
        fill(fc.species_configs['iirf_0'], fair_params_df.loc[iconf, 'iirf_0[CO2]'], specie='CO2')
        fill(fc.species_configs['iirf_airborne'], fair_params_df.loc[iconf, 'iirf_airborne[CO2]'], specie='CO2')
        fill(fc.species_configs['iirf_uptake'], fair_params_df.loc[iconf, 'iirf_uptake[CO2]'], specie='CO2')
        fill(fc.species_configs['iirf_temperature'], fair_params_df.loc[iconf, 'iirf_temperature[CO2]'], specie='CO2')
        fill(fc.species_configs["forcing_scale"], fair_params_df.loc[iconf, "forcing_scale[CO2]"], specie='CO2')
        fill(fc.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
        fill(fc.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
        fill(fc.species_configs['baseline_concentration'], 273.021047, specie='N2O')
        fill(fc.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
        fill(fc.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
        fill(fc.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')

        initialise(fc.concentration, fc.species_configs['baseline_concentration'])
        initialise(fc.forcing, 0)
        initialise(fc.temperature, 0)
        initialise(fc.airborne_emissions, 0)
        initialise(fc.cumulative_emissions, 0)
        initialise(fc.ocean_heat_content_change, 0)
        fc.run(progress=False)
        
        temperature[ce][:, idx][:branch[ce][idx]+1] = fc.temperature[:, 0, 0, 0]
        ohc[ce][:, idx][:branch[ce][idx]+1] = fc.ocean_heat_content_change[:, 0, 0]
        toa[ce][:, idx][:branch[ce][idx]+1] = fc.toa_imbalance[:, 0, 0]
        carbon_uptake[ce][:, idx][:branch[ce][idx]+1] = fc.cumulative_emissions[:, 0, 0, 0] - fc.airborne_emissions[:, 0, 0, 0]
        concentration[ce][:, idx][:branch[ce][idx]+1] = fc.concentration[:, 0, 0, 0]
        erf[ce][:, idx][:branch[ce][idx]+1] = fc.forcing_sum[:, 0, 0]

        # Switch to emissions driven with zero emissions and run for 1010 years.
        fe = FAIR()
        fe.define_time(branch[ce][idx], branch[ce][idx]+1010, 1)
        fe.define_scenarios([f'1pct-branch-{ce}PgC'])
        fe.define_configs([iconf])
        fe.define_species(species, emis_properties)
        fe.allocate()
        fe.concentration.loc[dict(specie='CH4')] = 808.2490285
        fe.concentration.loc[dict(specie='N2O')] = 273.021047
        fe.emissions.loc[dict(specie='CO2')] = 0
        fe.fill_species_configs()
        fill(fe.climate_configs['ocean_heat_capacity'], fair_params_df.loc[iconf,'ocean_heat_capacity[0]':'ocean_heat_capacity[2]'])
        fill(fe.climate_configs['ocean_heat_transfer'], fair_params_df.loc[iconf,'ocean_heat_transfer[0]':'ocean_heat_transfer[2]'])
        fill(fe.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[iconf,'deep_ocean_efficacy'])
        fill(fe.climate_configs['gamma_autocorrelation'], fair_params_df.loc[iconf,'gamma_autocorrelation'])
        fill(fe.climate_configs['stochastic_run'], False)
        fill(fe.species_configs['iirf_0'], fair_params_df.loc[iconf, 'iirf_0[CO2]'], specie='CO2')
        fill(fe.species_configs['iirf_airborne'], fair_params_df.loc[iconf, 'iirf_airborne[CO2]'], specie='CO2')
        fill(fe.species_configs['iirf_uptake'], fair_params_df.loc[iconf, 'iirf_uptake[CO2]'], specie='CO2')
        fill(fe.species_configs['iirf_temperature'], fair_params_df.loc[iconf, 'iirf_temperature[CO2]'], specie='CO2')
        fill(fe.species_configs["forcing_scale"], fair_params_df.loc[iconf, "forcing_scale[CO2]"], specie='CO2')
        fill(fe.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
        fill(fe.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
        fill(fe.species_configs['baseline_concentration'], 273.021047, specie='N2O')
        fill(fe.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
        fill(fe.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
        fill(fe.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')
        initialise(fe.concentration, fc.concentration[-1, ...])
        initialise(fe.forcing, fc.forcing[-1, ...])
        initialise(fe.temperature, fc.temperature[-1, ...])
        initialise(fe.airborne_emissions, fc.airborne_emissions[-1, ...])
        initialise(fe.cumulative_emissions, fc.cumulative_emissions[-1, ...])
        initialise(fe.alpha_lifetime, fc.alpha_lifetime[-1, ...])
        initialise(fe.ocean_heat_content_change, fc.ocean_heat_content_change[-1, ...])
        fe.gas_partitions=fc.gas_partitions.copy()
        fe.run(progress=False)

        temperature[ce][:, idx][branch[ce][idx] : branch[ce][idx] + 1011] = fe.temperature[:, 0, 0, 0]
        ohc[ce][:, idx][branch[ce][idx] : branch[ce][idx] + 1011] = fe.ocean_heat_content_change[:, 0, 0]
        toa[ce][:, idx][branch[ce][idx] : branch[ce][idx] + 1011] = fe.toa_imbalance[:, 0, 0]
        carbon_uptake[ce][:, idx][branch[ce][idx] : branch[ce][idx] + 1011] = fe.cumulative_emissions[:, 0, 0, 0] - fe.airborne_emissions[:, 0, 0, 0]
        concentration[ce][:, idx][branch[ce][idx] : branch[ce][idx] + 1011] = fe.concentration[:, 0, 0, 0]
        erf[ce][:, idx][branch[ce][idx] : branch[ce][idx] + 1011] = fe.forcing_sum[:, 0, 0]
    
    ecs[idx] = fe.ebms.ecs.data[0]
    tcr[idx] = fe.ebms.tcr.data[0]

In [ ]:
branch[1000][:8]

In [ ]:
pl.plot(temperature[750], color='blue');
pl.plot(temperature[1000], color='green');
pl.plot(temperature[2000], color='red');

In [ ]:
pl.plot(ohc[750], color='blue');
pl.plot(ohc[1000], color='green');
pl.plot(ohc[2000], color='red');

In [ ]:
pl.plot(toa[750], color='blue');
pl.plot(toa[1000], color='green');
pl.plot(toa[2000], color='red');

In [ ]:
pl.plot(carbon_uptake[750], color='blue');
pl.plot(carbon_uptake[1000], color='green');
pl.plot(carbon_uptake[2000], color='red');

In [ ]:
pl.plot(erf[750], color='blue');
pl.plot(erf[1000], color='green');
pl.plot(erf[2000], color='red');

In [ ]:
pl.plot(concentration[750], color='blue');
pl.plot(concentration[1000], color='green');
pl.plot(concentration[2000], color='red');

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature = (["scenario", "timebounds", "config"], np.array([temperature[750], temperature[1000], temperature[2000]])),
        ocean_heat_content = (["scenario", "timebounds", "config"], np.array([ohc[750], ohc[1000], ohc[2000]])),
        toa_imbalance = (["scenario", "timebounds", "config"], np.array([toa[750], toa[1000], toa[2000]])),
        carbon_uptake = (["scenario", "timebounds", "config"], np.array([carbon_uptake[750], carbon_uptake[1000], carbon_uptake[2000]])),
        effective_radiative_forcing = (["scenario", "timebounds", "config"], np.array([erf[750], erf[1000], erf[2000]])),
        concentration = (["scenario", "timebounds", "config"], np.array([concentration[750], concentration[1000], concentration[2000]])),
        branch_point = (["scenario", "config"], np.array([branch[750], branch[1000], branch[2000]])),
        ecs = (["config"], ecs),
        tcr = (["config"], tcr),
    ),
    coords = dict(
        timebounds = np.arange(1150),
        config = fair_params_df.index,
        scenario = ['1pct-branch-750PgC', '1pct-branch-1000PgC', '1pct-branch-2000PgC'],
    ),
)

In [ ]:
ds

In [ ]:
ds.to_netcdf('../data/1pct-branch.nc')